In [22]:
import os 
os.chdir("/home/om/Desktop/dl-project")

%pwd

'/home/om/Desktop/dl-project'

In [ ]:
from dataclasses import dataclass
from pathlib import Path
@dataclass(frozen=True)
class EvaluationConfig:
    """
    Configuration for evaluation of the model.
    """
    path_of_model: Path
    training_data_path: Path
    all_params: dict
    params_image_size: list
    params_batch_size: int
    

In [24]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories
import tensorflow as tf

In [25]:
# from cnnClassifier.entity.config_entity import EvaluationConfig

class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model=Path(self.config.train_model.model_path),
            training_data_path=Path(
                os.path.join(
                    self.config.data_ingestion.unzip_dir,
                    "kidney-CT_Scan/CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone"
                )
            ),
            all_params=dict(self.params.prepare_base_model),
            params_image_size=self.params.prepare_base_model.IMAGE_SIZE,
            params_batch_size=self.params.prepare_base_model.BATCH_SIZE
        )
        return eval_config

In [26]:
import tensorflow as tf
from cnnClassifier import logger
# from cnnClassifier.entity.config_entity import EvaluationConfig

class ModelEvaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    def load_model(self):
        logger.info(f"Loading model from {self.config.path_of_model}")
        model = tf.keras.models.load_model(self.config.path_of_model, compile=False)
        model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
        logger.info("Model loaded successfully.")
        return model

    def get_data_generator(self):
        from tensorflow.keras.preprocessing import image_dataset_from_directory
        IMG_SIZE = self.config.params_image_size[:2]
        BATCH_SIZE = self.config.params_batch_size
        DATA_DIR = self.config.training_data_path

        val_ds = image_dataset_from_directory(
            DATA_DIR,
            validation_split=0.2,
            subset="validation",
            seed=42,
            image_size=IMG_SIZE,
            batch_size=BATCH_SIZE,
            label_mode="categorical"
        )
        return val_ds

    def evaluate(self):
        model = self.load_model()
        val_ds = self.get_data_generator()
        logger.info("Evaluating model on validation set...")
        results = model.evaluate(val_ds, return_dict=True)
        logger.info(f"Evaluation results: {results}")
        return results

In [ ]:
# from cnnClassifier.config.configuration import ConfigurationManager
# from cnnClassifier.components.model_evaluation import ModelEvaluation
from cnnClassifier import logger
import mlflow
import dagshub

STAGE_NAME = "Model Evaluation"

class ModelEvaluationPipeline:
    """
    Pipeline for evaluating the trained model and logging metrics to MLflow.
    """

    def __init__(self, config=None):
        self.config = config or ConfigurationManager()

    def main(self):
        try:
            # Initialize dagshub/MLflow connection
            dagshub.init(
                repo_owner='ompandey0703',
                repo_name='kidney-classification',
                mlflow=True
            )

            eval_config = self.config.get_evaluation_config()
            evaluator = ModelEvaluation(config=eval_config)
            results = evaluator.evaluate()
            logger.info(f"Model evaluation results: {results}")

            # Log metrics to MLflow
            with mlflow.start_run(run_name="model-evaluation"):
                 # Log parameters (example: model name, batch size, image size, etc.)
                mlflow.log_param("model_name", eval_config.all_params.get("MODEL_NAME", "unknown"))
                mlflow.log_param("batch_size", eval_config.params_batch_size)
                mlflow.log_param("image_size", eval_config.params_image_size)
                # Log all params from all_params dict
                for k, v in eval_config.all_params.items():
                    mlflow.log_param(k, v)
    
                for k, v in results.items():
                    mlflow.log_metric(k, v)

        except Exception as e:
            logger.exception(f"Exception occurred during model evaluation: {e}")
            raise e

if __name__ == "__main__":
    try:
        logger.info(f">>>>>> {STAGE_NAME} started <<<<<<")
        config_manager = ConfigurationManager()
        pipeline = ModelEvaluationPipeline(config=config_manager)
        pipeline.main()
        logger.info(f">>>>>> {STAGE_NAME} completed <<<<<<")
    except Exception as e:
        logger.exception(f"An error occurred in the {STAGE_NAME} stage: {e}")
        raise e

2025-07-07 11:59:25,829 - INFO - 4165767654 - >>>>>> Model Evaluation started <<<<<<
2025-07-07 11:59:25,837 - INFO - common - Created directory: artifacts
2025-07-07 11:59:27,235 - INFO - _client - HTTP Request: GET https://dagshub.com/api/v1/repos/ompandey0703/kidney-classification "HTTP/1.1 200 OK"


Initialized MLflow to track repo "ompandey0703/kidney-classification"

2025-07-07 11:59:27,246 - INFO - helpers - Initialized MLflow to track repo "ompandey0703/kidney-classification"


Repository ompandey0703/kidney-classification initialized!

2025-07-07 11:59:27,248 - INFO - helpers - Repository ompandey0703/kidney-classification initialized!
2025-07-07 11:59:27,249 - INFO - 2103913484 - Loading model from artifacts/train_model/model.h5
2025-07-07 11:59:27,882 - INFO - 2103913484 - Model loaded successfully.
Found 12446 files belonging to 4 classes.
Using 2489 files for validation.
2025-07-07 11:59:28,365 - INFO - 2103913484 - Evaluating model on validation set...
78/78 ━━━━━━━━━━━━━━━━━━━━ 338s 4s/step - accuracy: 0.9935 - loss: 0.0916
2025-07-07 12:05:06,611 - INFO - 2103913484 - Evaluation results: {'accuracy': 0.9959823489189148, 'loss': 0.06957823038101196}
2025-07-07 12:05:06,612 - INFO - 4165767654 - Model evaluation results: {'accuracy': 0.9959823489189148, 'loss': 0.06957823038101196}
2025-07-07 12:05:06,748 - WARNING - connectionpool - Retrying (Retry(total=4, connect=5, read=4, redirect=5, status=5)) after connection broken by 'RemoteDisconnected('Remote end closed connection without response')': /ompandey0703/ki

In [28]:
import tensorflow as tf
print(tf.__version__)

2.18.0
